In [ ]:
### 降低维度（扔掉一些维度）
#### 维度太多，如果维度之间有关联可能影响分析的结果

In [1]:
import pandas
from sklearn.feature_selection import VarianceThreshold
df = pandas.read_csv('Data/customer_behavior.csv')
df

ModuleNotFoundError: No module named 'sklearn'

In [ ]:
X = df[['bachelor','gender', 'age', 'salary']]
## 剔除低于阈值的特征，减少特征
sel = VarianceThreshold()
X_val = sel.fit_transform(X)
#X_val

# sel.get_support()得到布尔值
names = df.columns[sel.get_support()]
names

In [ ]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
X = df[['bachelor','gender', 'age', 'salary']]
y = df['purchased'].values
## 选择最相关的特征（2个）
clf = SelectKBest(chi2, k=2)
clf.fit(X,y)
print(clf.scores_)

In [ ]:
X_new = clf.fit_transform(X,y)
print(X_new)

In [ ]:
from sklearn.cross_validation import cross_val_score, ShuffleSplit
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.cross_validation import cross_val_score
clf = DecisionTreeClassifier(random_state=123)
scores = []
## 通过决策树分类器对特征进行评估，找出对分类性能贡献最大的特征
## X_val.shape[1]，shape是二维数组，这里取的是列数，也就是特征的数量
for i in range(X_val.shape[1]):
    # 交叉验证（模型，特征向量（一次只取一个），目标向量，评判标准，交叉策略）
    # cv(样本数，交叉验证次数，测试集占总样本的比例)
    score = cross_val_score(clf, X_val[:, i:i+1], y, scoring="accuracy",
                              cv=ShuffleSplit(len(X_val), 3, .3))
    scores.append((round(np.mean(score), 3), names[i]))
print(sorted(scores, reverse=True))

In [ ]:
scores_chi2

In [ ]:
from sklearn.feature_selection import RFE
from sklearn.svm import SVC

clf = SVC(kernel='linear')

# 选择一个最好的特征
rfe = RFE(clf, n_features_to_select=1)
rfe.fit(X_val,y)

# rfe.ranking_每个特征的排名信息
for x in rfe.ranking_:
    print(names[x-1], rfe.ranking_[x-1])

In [ ]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(n_estimators=10, random_state=123)

clf.fit(X_val, y)

names, clf.feature_importances_
for feature in zip(names, clf.feature_importances_):
    print(feature)

In [ ]:
clf.feature_importances_

In [ ]:
range(1, names)

In [ ]:
import matplotlib.pyplot as plt
plt.title('Feature Importance')
plt.bar(range(0, len(names)), clf.feature_importances_)
plt.xticks(range(0,len(names)), names)
plt.show()